# utils 

> utils for this project

In [ ]:
# | default_exp utils

In [ ]:
# | hide
# from nbdev.showdoc import *

## Dependency Parser

By default nbdev uses `requirements` and `dev_requirements` attributes form `settings.ini` to install dependencies.

However, we want to use `Pipfile` file to install dependencies.
 
To do so, we need to override the default behavior of nbdev. We do this by creating a custom `DependencyParser` class.

 Example of setuptools expected format for dependencies:
 
 ```python
     install_requires=[
        'docutils',
        'BazSpam ==1.1',
        "black['jupyter'] ==22.12.0",
    ],
```

In [ ]:
# | export
from pipfile import Pipfile


class DependencyParser:
    def __init__(self, path_to_pipfile: str):
        self.pipfile = Pipfile.load(filename=path_to_pipfile)

    def pipenv_to_setuptools(self, package_name: str, contraints: str | dict) -> str:
        if contraints == "*":
            res = f"{package_name}"
        elif (contraints != "*") and isinstance(contraints, str):
            res = f"{package_name} {contraints}"
        elif isinstance(contraints, dict):
            if extras := contraints.get("extras"):
                extras = [f"'{e}'" for e in extras]  # add ' to extras in list
                package_name = f"{package_name}[{','.join(extras)}]"
                res = f"{package_name} {contraints['version']}"
            else:
                res = f"{package_name} {contraints['version']}"
        else:
            raise ValueError(f"Could not parse {package_name}, {contraints}")

        return res

    @property
    def min_python(self) -> str:
        return self.pipfile.data["_meta"]["requires"]["python_version"]

    @property
    def requirements(self):
        return [
            self.pipenv_to_setuptools(k, v)
            for k, v in self.pipfile.data["default"].items()
        ]

    @property
    def dev_requirements(self):
        return [
            self.pipenv_to_setuptools(k, v)
            for k, v in self.pipfile.data["develop"].items()
        ]

In [ ]:
# test DependencyParser on test data
deps_parsed = DependencyParser("../test_data/utils/Pipfile")
assert deps_parsed.min_python == "3.6"
assert set(deps_parsed.requirements) == set(["simple ==2.28"])
assert set(deps_parsed.dev_requirements) == set(
    ["nested['jupyter'] ==22.12.0", "flexible ==1.*", "star", "gt >2.20", "gte >=2.20"]
)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()